# Cross Validation

### Beatriz Loureiro 
### Giovanni Dalvi
### Richard Sousa

In [2]:
import sys
import importlib
sys.path.append('../')
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import math
from sklearn.model_selection import cross_val_score, StratifiedKFold


from src.finance_ml.data_preparation.data_preparation import DataLoader
from src.finance_ml.cross_validation.cross_validation import CrossValidation

C:\Users\Richa\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [35]:
# Defining time_index_col (must be the same column in all inputs) and keep_cols refering to the columns that will remain in the dataset
dataloader = DataLoader(time_index_col= 'DATE', 
                    keep_cols = ['VOLUME','OPEN', 'HIGHT', 'LOW', 'CLOSE', 'VW','TRANSACTIONS'])

In [43]:
# Example Files
fname_AAPL = 'equities/AAPL_2020-04-07_2022-04-06.parquet'

# No. of Records from example dataset
N = 30000

In [44]:
# Dataset chosen in this simulation
ticker = 'AAPL'
fname = fname_AAPL

In [45]:
# loading assets into to an unique df
df = dataloader.load_dataset({ticker:'../data/'+fname}).iloc[:N]
display(df)

,AAPL_VOLUME,AAPL_OPEN,AAPL_HIGHT,AAPL_LOW,AAPL_CLOSE,AAPL_VW,AAPL_TRANSACTIONS
DATE,,,,,,,
2020-04-07 12:17:00,14256.0,67.3500,67.3625,67.3275,67.3375,67.3454,45
2020-04-07 12:18:00,16552.0,67.3501,67.3625,67.3250,67.3250,67.3407,57
2020-04-07 12:19:00,80172.0,67.3250,67.4475,67.3250,67.4225,67.3937,149
2020-04-07 12:20:00,183740.0,67.4475,67.4700,67.3125,67.3700,67.3671,159
2020-04-07 12:21:00,35372.0,67.3500,67.3850,67.2975,67.2975,67.3469,102
...,...,...,...,...,...,...,...
2020-06-05 15:42:00,282372.0,82.1317,82.1725,82.0925,82.1168,82.1329,769
2020-06-05 15:43:00,195168.0,82.1125,82.1250,82.0925,82.1100,82.1059,533
2020-06-05 15:44:00,290188.0,82.1075,82.2450,82.1050,82.2400,82.1737,756


In [7]:
X = df.drop('AAPL_VW', axis=1)
Y = df['AAPL_VW']

In [17]:
import warnings
warnings.filterwarnings('ignore')

CV = CrossValidation(overlap=0, embargo_rate = 0)

for k in range(3):
    alpha = (k+1)/10
    scores = CV.cross_validation_score(X, Y, Lasso(alpha=alpha), 'mean_absolute_error')
    print('\nAlpha: ', alpha,'\nlista_scores: ', scores, '\nmédia score: ', np.mean(scores))
    scores_classico = -cross_val_score(Lasso(alpha=alpha), X, Y, cv=5, scoring='neg_mean_absolute_error')
    print('\nCross-Validation normal do sklearn: ', scores_classico, '\nmédia score: ', np.mean(scores_classico))


Alpha:  0.1 
lista_scores:  [0.04557962173132069, 0.034404094025028656, 0.024150088408072572, 0.033862093779789014, 0.002846443506758069] 
média score:  0.028168468290193798

Cross-Validation normal do sklearn:  [0.04557962 0.03440409 0.02415009 0.03386209 0.04543752] 
média score:  0.03668668350519265

Alpha:  0.2 
lista_scores:  [0.07973677232870408, 0.05747970378464099, 0.02810419523200188, 0.04947069123644686, 0.00790305623013826] 
média score:  0.04453888376238642

Cross-Validation normal do sklearn:  [0.07973677 0.0574797  0.0281042  0.04947069 0.08389876] 
média score:  0.05973802457579495

Alpha:  0.3 
lista_scores:  [0.11703186265072893, 0.08327156383442942, 0.03303163135784793, 0.06734967204673023, 0.016311252390461013] 
média score:  0.0633991964560395

Cross-Validation normal do sklearn:  [0.11703186 0.08327156 0.03303163 0.06734967 0.12385532] 
média score:  0.0849080098343474
